In [5]:
from typing import TypedDict, List, Annotated, Optional
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [6]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [20]:
class JobAgent(TypedDict, total=False):
    
    #Initial user input
    name: str
    job_title: str
    experience: int   # 0 = fresher
    pdf_path: str
    jd: str
    email: Optional[str]
    linkedin: Optional[str]

    #Initial user input
    resume_text: str
    sections : List[str]
    missing_fields: List[str]
    score: float
    feedback: str
    interview_questions: List[str]
    recommended_courses: List[str]
    job_links: List[str]

MAKING ALL THE FUNCTION DEFINED TO RE-MAP THE GRAPH

STEP 1 = EXTRACTION
#here we will load the pdf and extract all the 
#text and store in any list formate
#this function will return a string which is here text

In [19]:
from langchain_community.document_loaders import PyPDFLoader
def extraction(state : JobAgent) -> JobAgent:
    pdf_path = state["pdf_path"]
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text = ""
    for doc in docs:
        text += doc.page_content + "\n"
    
    state["resume_text"] = text
    return state
    

STEP 2

In [21]:
import re
def preprocess(state : JobAgent) -> JobAgent:
    text = state.get("resume_text")
    headings = [
        "SUMMARY", "OBJECTIVE",
        "SKILLS", "TECHNICALSKILLS",
        "EDUCATION",
        "EXPERIENCE", "WORKEXPERIENCE",
        "PROJECTS", "PROJECT",
        "CERTIFICATIONS",
        "ACHIEVEMENTS",
        "INTERNSHIPS",
        "LANGUAGES"
    ]
    required_words = ["github", "gmail.com", "linkedin", "leetcode"]

    #cheking required
    missing = []
    lower_text = text.lower()
    
    for word in required_words:
        if word not in lower_text:
            missing.append(word)
    
    #section splitting
    sections = {"OTHER" : ""}
    current = "OTHER"

    for line in text.split("/n"):
        line = line.strip()
        if not line :
            continue

        normalised = re.sub(r"[^A-Za-z]", "", line).upper()

        if normalised in headings:
            current = normalised
            sections[current] = ""
        else :
            sections[current] += line + "\n"

    state["missing_fields"] = missing
    state["sections"] = sections
    
    return state
    


In [ ]:
from langchain_
def missing(state : JobAgent) -> JobAgent:
    prompt = Chat

NODES

we can store all this info in cache and if in future 
the user give jd we will give me perceatge how fit he is or not

In [ ]:
graph = StateGraph(JobAgent)
#NODE

graph.add_node('extraction', extraction)
graph.add_node('preprocess', preprocess)
#major part
graph.add_node('missing', missing)
graph.add_node('assestment', assesment)
#graph.add_node('voice', voice)
graph.add_node('interview', interview)
#resources
graph.add_node('dsa', dsa)
graph.add_node('job_sites', job_sites)
graph.add_node('courses', courses)

#summay and feedback
graph.add_node('jd_feedback', jd_feedback)

#now if the feedback is good we will mail the recutier and messge in linkdn
#update in excell so that he can track 

EDGES

In [ ]:
#EDGE
graph.add_edge(START, 'extraction')
graph.add_edge('extraction', 'preprocess')
graph.add_edge('preprocess', 'missing')
graph.add_edge('missing', 'assestment')
#here we can add a condition edges where if he/she get 70% right he will 
#not get interview question he will direcly get interviwe and dsa 
graph.add_edge('assestment', 'interview')
graph.add_edge('interview', 'dsa')
graph.add_edge('dsa', 'job_sites')
graph.add_edge('job_sites', 'courses')
graph.add_edge('courses', 'jd_feedback')
graph.add_edge("jd_feedback", END)

workflow = graph.compile()